
# Daily Python DSA – LeetCode‑style Practice Notebook

**How this works**
- Run the **Session Runner** cell to get 3–5 random problems (customizable).
- Each problem shows a prompt and a code cell with a starter template.
- A timer starts when you run the problem cell; stop it with `stop_timer()`.
- Use the provided tests to validate your solution.
- Results are logged to `/mnt/data/dsa_practice_log.csv`, including:
  problem id, date, duration (sec), pass/fail, and next review date.

**Spaced repetition (light Leitner)**
- If you pass fast (`< 10 min`): review in **7 days**.
- If you pass but slow (`10–25 min`): review in **3 days**.
- If you fail tests: review **tomorrow**.

**Tip**: Treat it like a real interview—talk through ideas, write clean code, analyze time/space.


In [ ]:

# === Helpers: timer, logger, scheduler ===
from __future__ import annotations
import time, csv, os, random, math, datetime as dt
from dataclasses import dataclass
from typing import List, Dict, Tuple, Optional
from collections import deque, OrderedDict
import heapq

_TIMER_START = None

def start_timer():
    global _TIMER_START
    _TIMER_START = time.time()
    print("⏱️ Timer started.")

def stop_timer() -> float:
    global _TIMER_START
    if _TIMER_START is None:
        print("Timer was not started.")
        return 0.0
    dur = time.time() - _TIMER_START
    _TIMER_START = None
    print(f"⏱️ Stopped. Duration: {dur:.1f}s")
    return dur

LOG_PATH = "/mnt/data/dsa_practice_log.csv"

def _ensure_log():
    if not os.path.exists(LOG_PATH):
        with open(LOG_PATH, "w", newline="") as f:
            w = csv.writer(f)
            w.writerow(["date","problem_id","duration_sec","result","next_review_date"])

def _next_review(duration_sec: float, passed: bool) -> str:
    today = dt.date.today()
    if not passed:
        nd = today + dt.timedelta(days=1)
    else:
        if duration_sec < 600:      # <10 min
            nd = today + dt.timedelta(days=7)
        elif duration_sec < 1500:   # <25 min
            nd = today + dt.timedelta(days=3)
        else:
            nd = today + dt.timedelta(days=1)
    return nd.isoformat()

def log_result(problem_id: str, duration_sec: float, passed: bool):
    _ensure_log()
    next_date = _next_review(duration_sec, passed)
    with open(LOG_PATH, "a", newline="") as f:
        w = csv.writer(f)
        w.writerow([dt.date.today().isoformat(), problem_id, f"{duration_sec:.1f}", "PASS" if passed else "FAIL", next_date])
    print(f"📝 Logged: {problem_id} | {'PASS' if passed else 'FAIL'} | next review {next_date} -> {LOG_PATH}")


In [ ]:
# Registry of problems

In [ ]:
PROBLEMS = {
    "P1_two_sum": {"desc": 'Two Sum — return indices i, j such that nums[i] + nums[j] == target (exactly one solution). Time O(n).', "template": 'def two_sum(nums, target):\n    # TODO: implement\n    pass\n', "tests": '# --- Tests ---\nassert tuple(sorted(two_sum([2,7,11,15], 9))) == (0,1)\nassert tuple(sorted(two_sum([3,2,4], 6))) == (1,2)\nprint("OK")\n', "solution": 'def two_sum(nums, target):\n    seen = {}\n    for i, x in enumerate(nums):\n        need = target - x\n        if need in seen:\n            return (seen[need], i)\n        seen[x] = i\n    raise ValueError("No solution")\n'},
    "P2_longest_substring": {"desc": 'Longest Substring Without Repeating Characters — return the length. Sliding window O(n).', "template": 'def length_of_longest_substring(s):\n    # TODO: implement\n    pass\n', "tests": '# --- Tests ---\nassert length_of_longest_substring("abcabcbb") == 3\nassert length_of_longest_substring("bbbbb") == 1\nassert length_of_longest_substring("pwwkew") == 3\nprint("OK")\n', "solution": 'def length_of_longest_substring(s):\n    start = 0\n    best = 0\n    last = {}\n    for i, ch in enumerate(s):\n        if ch in last and last[ch] >= start:\n            start = last[ch] + 1\n        last[ch] = i\n        best = max(best, i - start + 1)\n    return best\n'},
    "P3_merge_intervals": {"desc": 'Merge Intervals — merge overlaps. Sort by start. O(n log n).', "template": 'def merge_intervals(intervals):\n    # TODO: implement\n    pass\n', "tests": '# --- Tests ---\nassert merge_intervals([[1,3],[2,6],[8,10],[15,18]]) == [[1,6],[8,10],[15,18]]\nassert merge_intervals([]) == []\nprint("OK")\n', "solution": 'def merge_intervals(intervals):\n    if not intervals:\n        return []\n    intervals.sort(key=lambda x: x[0])\n    out = [intervals[0]]\n    for s, e in intervals[1:]:\n        if s <= out[-1][1]:\n            out[-1][1] = max(out[-1][1], e)\n        else:\n            out.append([s, e])\n    return out\n'},
    "P4_kth_largest": {"desc": 'Kth Largest Element in an Array — min-heap of size k. O(n log k).', "template": 'def kth_largest(nums, k):\n    # TODO: implement\n    pass\n', "tests": '# --- Tests ---\nassert kth_largest([3,2,1,5,6,4], 2) == 5\nassert kth_largest([3,2,3,1,2,4,5,5,6], 4) == 4\nprint("OK")\n', "solution": 'import heapq\ndef kth_largest(nums, k):\n    h = []\n    for x in nums:\n        if len(h) < k:\n            heapq.heappush(h, x)\n        else:\n            if x > h[0]:\n                heapq.heapreplace(h, x)\n    return h[0]\n'},
    "P5_reverse_list": {"desc": 'Reverse Singly Linked List — iterative O(n), O(1) space.', "template": 'class ListNode:\n    def __init__(self, val=0, next=None):\n        self.val = val\n        self.next = next\n\ndef reverse_list(head):\n    # TODO: implement\n    pass\n', "tests": '# --- Tests ---\na = ListNode(1, ListNode(2, ListNode(3)))\nr = reverse_list(a)\nassert (r.val, r.next.val, r.next.next.val) == (3,2,1)\nprint("OK")\n', "solution": 'class ListNode:\n    def __init__(self, val=0, next=None):\n        self.val = val\n        self.next = next\n\ndef reverse_list(head):\n    prev = None\n    curr = head\n    while curr:\n        nxt = curr.next\n        curr.next = prev\n        prev = curr\n        curr = nxt\n    return prev\n'},
    "P6_level_order": {"desc": 'Binary Tree Level Order Traversal — BFS using deque.', "template": 'from collections import deque\nclass TreeNode:\n    def __init__(self, val=0, left=None, right=None):\n        self.val = val\n        self.left = left\n        self.right = right\n\ndef level_order(root):\n    # TODO: implement\n    pass\n', "tests": '# --- Tests ---\nroot = TreeNode(1, TreeNode(2), TreeNode(3, TreeNode(4), None))\nassert level_order(root) == [[1],[2,3],[4]]\nprint("OK")\n', "solution": 'from collections import deque\nclass TreeNode:\n    def __init__(self, val=0, left=None, right=None):\n        self.val = val\n        self.left = left\n        self.right = right\n\ndef level_order(root):\n    if not root:\n        return []\n    q = deque([root])\n    out = []\n    while q:\n        sz = len(q)\n        lvl = []\n        for _ in range(sz):\n            node = q.popleft()\n            lvl.append(node.val)\n            if node.left: q.append(node.left)\n            if node.right: q.append(node.right)\n        out.append(lvl)\n    return out\n'},
    "P7_num_islands": {"desc": 'Number of Islands — DFS flood fill on grid.', "template": 'def num_islands(grid):\n    # TODO: implement\n    pass\n', "tests": '# --- Tests ---\ng = [list(\'11000\'),list(\'11000\'),list(\'00100\'),list(\'00011\')]\nassert num_islands(g) == 3\nprint("OK")\n', "solution": "def num_islands(grid):\n    if not grid or not grid[0]:\n        return 0\n    m, n = len(grid), len(grid[0])\n    def dfs(r, c):\n        if r < 0 or c < 0 or r >= m or c >= n or grid[r][c] != '1':\n            return\n        grid[r][c] = '#'\n        dfs(r+1, c); dfs(r-1, c); dfs(r, c+1); dfs(r, c-1)\n    count = 0\n    for r in range(m):\n        for c in range(n):\n            if grid[r][c] == '1':\n                count += 1\n                dfs(r, c)\n    return count\n"},
    "P8_coin_change": {"desc": 'Coin Change — min coins DP, return -1 if impossible.', "template": 'def coin_change(coins, amount):\n    # TODO: implement\n    pass\n', "tests": '# --- Tests ---\nassert coin_change([1,2,5], 11) == 3\nassert coin_change([2], 3) == -1\nprint("OK")\n', "solution": 'def coin_change(coins, amount):\n    INF = amount + 1\n    dp = [0] + [INF] * amount\n    for a in range(1, amount + 1):\n        for c in coins:\n            if c <= a:\n                dp[a] = min(dp[a], dp[a-c] + 1)\n    return dp[amount] if dp[amount] != INF else -1\n'},
    "P9_permute": {"desc": 'Permutations — backtracking, O(n * n!).', "template": 'def permute(nums):\n    # TODO: implement\n    pass\n', "tests": '# --- Tests ---\nres = permute([1,2,3])\nexp = sorted([[1,2,3],[1,3,2],[2,1,3],[2,3,1],[3,1,2],[3,2,1]])\nassert sorted(res) == exp\nprint("OK")\n', "solution": 'def permute(nums):\n    res = []\n    used = [False]*len(nums)\n    path = []\n    def backtrack():\n        if len(path) == len(nums):\n            res.append(path.copy()); return\n        for i in range(len(nums)):\n            if used[i]: continue\n            used[i] = True\n            path.append(nums[i])\n            backtrack()\n            path.pop()\n            used[i] = False\n    backtrack()\n    return res\n'},
    "P10_lru_cache": {"desc": 'LRU Cache — OrderedDict O(1) get/put.', "template": 'from collections import OrderedDict\nclass LRUCache:\n    def __init__(self, capacity: int):\n        self.cap = capacity\n        self.od = OrderedDict()\n    def get(self, key: int) -> int:\n        # TODO: implement\n        pass\n    def put(self, key: int, value: int) -> None:\n        # TODO: implement\n        pass\n', "tests": '# --- Tests ---\nlru = LRUCache(2)\nlru.put(1,1); lru.put(2,2)\nassert lru.get(1) == 1\nlru.put(3,3)    # evicts 2\nassert lru.get(2) == -1\nlru.put(4,4)    # evicts 1\nassert (lru.get(1), lru.get(3), lru.get(4)) == (-1,3,4)\nprint("OK")\n', "solution": 'from collections import OrderedDict\nclass LRUCache:\n    def __init__(self, capacity: int):\n        self.cap = capacity\n        self.od = OrderedDict()\n    def get(self, key: int) -> int:\n        if key not in self.od:\n            return -1\n        self.od.move_to_end(key)\n        return self.od[key]\n    def put(self, key: int, value: int) -> None:\n        if key in self.od:\n            self.od.move_to_end(key)\n        self.od[key] = value\n        if len(self.od) > self.cap:\n            self.od.popitem(last=False)\n'},
}
print(f"Loaded {len(PROBLEMS)} problems.")


In [ ]:

# === Session Runner ===
# Configure how many random problems to practice in this session.
NUM_PROBLEMS = 4   # change to 3–5 for a quick daily session
SHOW_SOLUTIONS = False  # set True to preload solution cells

import random, datetime as dt

def pick_due_problems(k: int):
    r"""Prefer problems due today or overdue based on log; fallback to random."""
    path = LOG_PATH
    due = []
    seen = set()
    today = dt.date.today()
    if os.path.exists(path):
        import csv
        with open(path, newline="") as f:
            r = csv.DictReader(f)
            for row in r:
                pid = row["problem_id"]
                seen.add(pid)
                try:
                    nd = dt.date.fromisoformat(row["next_review_date"])
                except Exception:
                    nd = today
                if nd <= today:
                    due.append(pid)
    pool = list(PROBLEMS.keys())
    random.shuffle(pool)
    chosen = []
    # take due first
    for pid in due:
        if pid in pool and pid not in chosen:
            chosen.append(pid)
            if len(chosen) == k:
                break
    # fill with random
    for pid in pool:
        if len(chosen) == k:
            break
        if pid not in chosen:
            chosen.append(pid)
    return chosen[:k]

SESSION = pick_due_problems(NUM_PROBLEMS)
print("Today's set:", SESSION)


## Run each problem cell below (timer + template + tests)

### P1_two_sum: Two Sum — return indices i, j such that nums[i] + nums[j] == target (exactly one solution). Time O(n).

In [ ]:
# Problem: P1_two_sum
print("Problem ID: P1_two_sum  -> Two Sum — return indices i, j such that nums[i] + nums[j] == target (exactly one solution). Time O(n).")
start_timer()
def two_sum(nums, target):
    # TODO: implement
    pass


In [ ]:
# Run tests for P1_two_sum and log
_duration = stop_timer()
try:
    # --- Tests ---
assert tuple(sorted(two_sum([2,7,11,15], 9))) == (0,1)
assert tuple(sorted(two_sum([3,2,4], 6))) == (1,2)
print("OK")

    _passed = True
except AssertionError:
    print("❌ Tests failed.")
    _passed = False
log_result("P1_two_sum", _duration, _passed)


In [ ]:
# Solution for P1_two_sum (set SHOW_SOLUTIONS=True above to auto-run/inspect)
if SHOW_SOLUTIONS:
    print("Revealed solution for P1_two_sum:")
    print('def two_sum(nums, target):\n    seen = {}\n    for i, x in enumerate(nums):\n        need = target - x\n        if need in seen:\n            return (seen[need], i)\n        seen[x] = i\n    raise ValueError("No solution")\n')


### P2_longest_substring: Longest Substring Without Repeating Characters — return the length. Sliding window O(n).

In [ ]:
# Problem: P2_longest_substring
print("Problem ID: P2_longest_substring  -> Longest Substring Without Repeating Characters — return the length. Sliding window O(n).")
start_timer()
def length_of_longest_substring(s):
    # TODO: implement
    pass


In [ ]:
# Run tests for P2_longest_substring and log
_duration = stop_timer()
try:
    # --- Tests ---
assert length_of_longest_substring("abcabcbb") == 3
assert length_of_longest_substring("bbbbb") == 1
assert length_of_longest_substring("pwwkew") == 3
print("OK")

    _passed = True
except AssertionError:
    print("❌ Tests failed.")
    _passed = False
log_result("P2_longest_substring", _duration, _passed)


In [ ]:
# Solution for P2_longest_substring (set SHOW_SOLUTIONS=True above to auto-run/inspect)
if SHOW_SOLUTIONS:
    print("Revealed solution for P2_longest_substring:")
    print('def length_of_longest_substring(s):\n    start = 0\n    best = 0\n    last = {}\n    for i, ch in enumerate(s):\n        if ch in last and last[ch] >= start:\n            start = last[ch] + 1\n        last[ch] = i\n        best = max(best, i - start + 1)\n    return best\n')


### P3_merge_intervals: Merge Intervals — merge overlaps. Sort by start. O(n log n).

In [ ]:
# Problem: P3_merge_intervals
print("Problem ID: P3_merge_intervals  -> Merge Intervals — merge overlaps. Sort by start. O(n log n).")
start_timer()
def merge_intervals(intervals):
    # TODO: implement
    pass


In [ ]:
# Run tests for P3_merge_intervals and log
_duration = stop_timer()
try:
    # --- Tests ---
assert merge_intervals([[1,3],[2,6],[8,10],[15,18]]) == [[1,6],[8,10],[15,18]]
assert merge_intervals([]) == []
print("OK")

    _passed = True
except AssertionError:
    print("❌ Tests failed.")
    _passed = False
log_result("P3_merge_intervals", _duration, _passed)


In [ ]:
# Solution for P3_merge_intervals (set SHOW_SOLUTIONS=True above to auto-run/inspect)
if SHOW_SOLUTIONS:
    print("Revealed solution for P3_merge_intervals:")
    print('def merge_intervals(intervals):\n    if not intervals:\n        return []\n    intervals.sort(key=lambda x: x[0])\n    out = [intervals[0]]\n    for s, e in intervals[1:]:\n        if s <= out[-1][1]:\n            out[-1][1] = max(out[-1][1], e)\n        else:\n            out.append([s, e])\n    return out\n')


### P4_kth_largest: Kth Largest Element in an Array — min-heap of size k. O(n log k).

In [ ]:
# Problem: P4_kth_largest
print("Problem ID: P4_kth_largest  -> Kth Largest Element in an Array — min-heap of size k. O(n log k).")
start_timer()
def kth_largest(nums, k):
    # TODO: implement
    pass


In [ ]:
# Run tests for P4_kth_largest and log
_duration = stop_timer()
try:
    # --- Tests ---
assert kth_largest([3,2,1,5,6,4], 2) == 5
assert kth_largest([3,2,3,1,2,4,5,5,6], 4) == 4
print("OK")

    _passed = True
except AssertionError:
    print("❌ Tests failed.")
    _passed = False
log_result("P4_kth_largest", _duration, _passed)


In [ ]:
# Solution for P4_kth_largest (set SHOW_SOLUTIONS=True above to auto-run/inspect)
if SHOW_SOLUTIONS:
    print("Revealed solution for P4_kth_largest:")
    print('import heapq\ndef kth_largest(nums, k):\n    h = []\n    for x in nums:\n        if len(h) < k:\n            heapq.heappush(h, x)\n        else:\n            if x > h[0]:\n                heapq.heapreplace(h, x)\n    return h[0]\n')


### P5_reverse_list: Reverse Singly Linked List — iterative O(n), O(1) space.

In [ ]:
# Problem: P5_reverse_list
print("Problem ID: P5_reverse_list  -> Reverse Singly Linked List — iterative O(n), O(1) space.")
start_timer()
class ListNode:
    def __init__(self, val=0, next=None):
        self.val = val
        self.next = next

def reverse_list(head):
    # TODO: implement
    pass


In [ ]:
# Run tests for P5_reverse_list and log
_duration = stop_timer()
try:
    # --- Tests ---
a = ListNode(1, ListNode(2, ListNode(3)))
r = reverse_list(a)
assert (r.val, r.next.val, r.next.next.val) == (3,2,1)
print("OK")

    _passed = True
except AssertionError:
    print("❌ Tests failed.")
    _passed = False
log_result("P5_reverse_list", _duration, _passed)


In [ ]:
# Solution for P5_reverse_list (set SHOW_SOLUTIONS=True above to auto-run/inspect)
if SHOW_SOLUTIONS:
    print("Revealed solution for P5_reverse_list:")
    print('class ListNode:\n    def __init__(self, val=0, next=None):\n        self.val = val\n        self.next = next\n\ndef reverse_list(head):\n    prev = None\n    curr = head\n    while curr:\n        nxt = curr.next\n        curr.next = prev\n        prev = curr\n        curr = nxt\n    return prev\n')


### P6_level_order: Binary Tree Level Order Traversal — BFS using deque.

In [ ]:
# Problem: P6_level_order
print("Problem ID: P6_level_order  -> Binary Tree Level Order Traversal — BFS using deque.")
start_timer()
from collections import deque
class TreeNode:
    def __init__(self, val=0, left=None, right=None):
        self.val = val
        self.left = left
        self.right = right

def level_order(root):
    # TODO: implement
    pass


In [ ]:
# Run tests for P6_level_order and log
_duration = stop_timer()
try:
    # --- Tests ---
root = TreeNode(1, TreeNode(2), TreeNode(3, TreeNode(4), None))
assert level_order(root) == [[1],[2,3],[4]]
print("OK")

    _passed = True
except AssertionError:
    print("❌ Tests failed.")
    _passed = False
log_result("P6_level_order", _duration, _passed)


In [ ]:
# Solution for P6_level_order (set SHOW_SOLUTIONS=True above to auto-run/inspect)
if SHOW_SOLUTIONS:
    print("Revealed solution for P6_level_order:")
    print('from collections import deque\nclass TreeNode:\n    def __init__(self, val=0, left=None, right=None):\n        self.val = val\n        self.left = left\n        self.right = right\n\ndef level_order(root):\n    if not root:\n        return []\n    q = deque([root])\n    out = []\n    while q:\n        sz = len(q)\n        lvl = []\n        for _ in range(sz):\n            node = q.popleft()\n            lvl.append(node.val)\n            if node.left: q.append(node.left)\n            if node.right: q.append(node.right)\n        out.append(lvl)\n    return out\n')


### P7_num_islands: Number of Islands — DFS flood fill on grid.

In [ ]:
# Problem: P7_num_islands
print("Problem ID: P7_num_islands  -> Number of Islands — DFS flood fill on grid.")
start_timer()
def num_islands(grid):
    # TODO: implement
    pass


In [ ]:
# Run tests for P7_num_islands and log
_duration = stop_timer()
try:
    # --- Tests ---
g = [list('11000'),list('11000'),list('00100'),list('00011')]
assert num_islands(g) == 3
print("OK")

    _passed = True
except AssertionError:
    print("❌ Tests failed.")
    _passed = False
log_result("P7_num_islands", _duration, _passed)


In [ ]:
# Solution for P7_num_islands (set SHOW_SOLUTIONS=True above to auto-run/inspect)
if SHOW_SOLUTIONS:
    print("Revealed solution for P7_num_islands:")
    print("def num_islands(grid):\n    if not grid or not grid[0]:\n        return 0\n    m, n = len(grid), len(grid[0])\n    def dfs(r, c):\n        if r < 0 or c < 0 or r >= m or c >= n or grid[r][c] != '1':\n            return\n        grid[r][c] = '#'\n        dfs(r+1, c); dfs(r-1, c); dfs(r, c+1); dfs(r, c-1)\n    count = 0\n    for r in range(m):\n        for c in range(n):\n            if grid[r][c] == '1':\n                count += 1\n                dfs(r, c)\n    return count\n")


### P8_coin_change: Coin Change — min coins DP, return -1 if impossible.

In [ ]:
# Problem: P8_coin_change
print("Problem ID: P8_coin_change  -> Coin Change — min coins DP, return -1 if impossible.")
start_timer()
def coin_change(coins, amount):
    # TODO: implement
    pass


In [ ]:
# Run tests for P8_coin_change and log
_duration = stop_timer()
try:
    # --- Tests ---
assert coin_change([1,2,5], 11) == 3
assert coin_change([2], 3) == -1
print("OK")

    _passed = True
except AssertionError:
    print("❌ Tests failed.")
    _passed = False
log_result("P8_coin_change", _duration, _passed)


In [ ]:
# Solution for P8_coin_change (set SHOW_SOLUTIONS=True above to auto-run/inspect)
if SHOW_SOLUTIONS:
    print("Revealed solution for P8_coin_change:")
    print('def coin_change(coins, amount):\n    INF = amount + 1\n    dp = [0] + [INF] * amount\n    for a in range(1, amount + 1):\n        for c in coins:\n            if c <= a:\n                dp[a] = min(dp[a], dp[a-c] + 1)\n    return dp[amount] if dp[amount] != INF else -1\n')


### P9_permute: Permutations — backtracking, O(n * n!).

In [ ]:
# Problem: P9_permute
print("Problem ID: P9_permute  -> Permutations — backtracking, O(n * n!).")
start_timer()
def permute(nums):
    # TODO: implement
    pass


In [ ]:
# Run tests for P9_permute and log
_duration = stop_timer()
try:
    # --- Tests ---
res = permute([1,2,3])
exp = sorted([[1,2,3],[1,3,2],[2,1,3],[2,3,1],[3,1,2],[3,2,1]])
assert sorted(res) == exp
print("OK")

    _passed = True
except AssertionError:
    print("❌ Tests failed.")
    _passed = False
log_result("P9_permute", _duration, _passed)


In [ ]:
# Solution for P9_permute (set SHOW_SOLUTIONS=True above to auto-run/inspect)
if SHOW_SOLUTIONS:
    print("Revealed solution for P9_permute:")
    print('def permute(nums):\n    res = []\n    used = [False]*len(nums)\n    path = []\n    def backtrack():\n        if len(path) == len(nums):\n            res.append(path.copy()); return\n        for i in range(len(nums)):\n            if used[i]: continue\n            used[i] = True\n            path.append(nums[i])\n            backtrack()\n            path.pop()\n            used[i] = False\n    backtrack()\n    return res\n')


### P10_lru_cache: LRU Cache — OrderedDict O(1) get/put.

In [ ]:
# Problem: P10_lru_cache
print("Problem ID: P10_lru_cache  -> LRU Cache — OrderedDict O(1) get/put.")
start_timer()
from collections import OrderedDict
class LRUCache:
    def __init__(self, capacity: int):
        self.cap = capacity
        self.od = OrderedDict()
    def get(self, key: int) -> int:
        # TODO: implement
        pass
    def put(self, key: int, value: int) -> None:
        # TODO: implement
        pass


In [ ]:
# Run tests for P10_lru_cache and log
_duration = stop_timer()
try:
    # --- Tests ---
lru = LRUCache(2)
lru.put(1,1); lru.put(2,2)
assert lru.get(1) == 1
lru.put(3,3)    # evicts 2
assert lru.get(2) == -1
lru.put(4,4)    # evicts 1
assert (lru.get(1), lru.get(3), lru.get(4)) == (-1,3,4)
print("OK")

    _passed = True
except AssertionError:
    print("❌ Tests failed.")
    _passed = False
log_result("P10_lru_cache", _duration, _passed)


In [ ]:
# Solution for P10_lru_cache (set SHOW_SOLUTIONS=True above to auto-run/inspect)
if SHOW_SOLUTIONS:
    print("Revealed solution for P10_lru_cache:")
    print('from collections import OrderedDict\nclass LRUCache:\n    def __init__(self, capacity: int):\n        self.cap = capacity\n        self.od = OrderedDict()\n    def get(self, key: int) -> int:\n        if key not in self.od:\n            return -1\n        self.od.move_to_end(key)\n        return self.od[key]\n    def put(self, key: int, value: int) -> None:\n        if key in self.od:\n            self.od.move_to_end(key)\n        self.od[key] = value\n        if len(self.od) > self.cap:\n            self.od.popitem(last=False)\n')
